# MP 6: Calibration and RANSAC

**Due date**: March 13, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. In this assignment, you should be switched over to a local install. 

To submit your assignment, perform the following:

1. Double-check that your programs run without error.
2. Send this file, all of your .py files and any other files you used in your programs on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).
3. If you are using any external libraries other than the ones that are indicated during the installation process, include a README file indicating which library and version you are using.  To be on the safe side, you should include a backup procedure until the graders verify that they are able to support use of that library.

## Problem 1: Plane extraction

We're going to perform intrinsics calibration of the SR300 depth camera by pointing it at perpendicular planes.  The first step is to extract the points that make up these planes.  To do so, in this problem we will implement a RANSAC-based plane extractor.  To keep your algorithms running at a reasonable speed, it will be helpful for you to learn array addressing in Numpy; running for loops in Python code is much, much slower.

The calibration color and depth images are stored in the `calibration/` folder, and they will be automatically loaded for you.  Specifically, we will be using the `depth_aligned_X.png` files.

### Problem 1.A

Implement a RANSAC algorithm under `extract_planes_ransac_a` in `planes.py`.  For `N` iterations, sample subsets of `m` points, fit planes, and examine how many inliers there are.  An inlier to a plane `(a,b,c,d)` is defined by $|(a,b,c)^T \mathbf{p} + d| < $ `inlier_tolerance`.

In this part, you will produce a naive implementation that outputs any inlier sets that are at least as large as `inlier_count`.   You will represent an inlier set using just a list of integers; the result from this function is a list of lists of integers.  The `fit_plane3` and `fit_plane` subroutines will allow you to fit these planes easily.

Running `planes.py` will perform this fitting for each of the depth images; the result will look something like this:

![Example output](example_output/planes_1a.png)


### Problem 1.B

The naive extractor obviously has problems, the main one being that the inlier sets overlap significantly.  Implement a refined version in `extract_planes_ransac_b` that 

1. Does not sample from candidate points that already belong to an extracted plane,
2. After the main RANSAC iteration, assigns points to the largest plane (in terms of # of inliers) for which they are an inlier, and 
3. Discards planes that have fewer than `inlier_count` points assigned to them, after step 2.

After uncommenting `PROBLEM='1b'`, the result should be much better, looking something like this:

![Example output](example_output/planes_1b.png)



### Problem 1.C

There are still some small errors near the boundary of the plane segmentation, which become more obvious when the inlier threshold is set larger.  In `extract_planes_ransac_c`, if a point can be an inlier for multiple planes, assign it to the one for which it is the best fit (i.e., the point-to-plane distance is lowest).  You might also modify the code to provide normals to your code, which you can use for even better assignments. 

After uncommenting `PROBLEM='1c'`, examine the boundaries between planes to ensure that you're getting reasonable results.   Tune the parameters to yield results that are as good as possible -- you might also want the parameters to depend on the scale of the input.  (Scan 9 is particularly challenging).

![Example output](example_output/planes_1c.png)

Then, for problem 2, click through each of the scans (or comment out the `vis.dialog()` call) to output the sets of planes for each image (the `planesets` variable).  This will output a file called `planesets.json` which you will include in your submission.

## Problem 2: Intrinsic calibration

Now, we will assume that we are given a default calibration for the camera taken from the manufacturer's specs, and use optimization to calibrate the $f_x, f_y, c_x, $ and $c_y$ intrinsic parameters.

### Problem 2.A

In `depth_calibration.py`, implement the `mutual_orthogonality()` function to determine how close to orthogonal are the planes determined for each scan.  You will keep the point indices belonging to each plane (the `planesets` variable) the same regardless of the setting of `fx,fy,cx,cy`, but you will change the points themselves.  In particular, see lines 165-169 of `rgbd.py` to see how the points are affected by these parameters. 

To measure orthogonality may choose to use an absolute value of the dot product (i.e., $|\cos \theta|$), dot product squared, angular error ($|90^\circ - \theta|$), or angular error squared.  To aggregate, you can either sum or average.  In any case, the result should be minimized when the normals of all planes detected for a scan are perfectly at 90 degrees.  (Note that for scan 11, two of the planes are parallel -- you should not penalize them.)

In the space below for written answers, describe the orthogonality metric you used and the method you used for aggregation.  Also, examine which of the scans produces the worst result(s), and check the visualization in `planes.py`.  Explain why those particular scans seemed problematic.

### Problem 2.B

Now, implement the `calibrate_intrinics_fxfy` function to optimize your metric only over the `fx,fy` variables, keeping `cx,cy` fixed.  Use the [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) function to minimize an objective function.  I suggest using the 'Nelder-Mead' method, which is a derivative-free method, and no more than 30 or 50 iterations.  Note that you will need to provide the optimization parameters as an array, write a 1-parameter wrapper function that accepts the array as input, and then interpret the results as an array.  

In the space below for written answers, report the initial and final parameter values, and the initial and final objective function values.  Did this seem to converge to reasonable values?  Compare with the parameters set in `calibration/color_intrinsics.json` (note: we're not using `depth_intrinsics.json` because the depth images have been aligned to the color camera).

### Problem 2.C

Now, implement the `calibrate_intrinics_all` function to optimize your metric over all four variables.   In the space below for written answers, report the initial and final parameter values, and the initial and final objective function values.  Did this seem to converge to reasonable values compared to `calibration/color_intrinsics.json`?  (Note that the Intel Realsense library refers to `cx, cy` as `ppx, ppy`.)  Explain why or why not.

Is this problem identifiable?  Give a rough justification for your answer (i.e., by verbally describing the geometry of the situation).

### Problem 2.D (IR2 section only)

Provide a rigorous mathematical justification for whether the full optimization is identifiable or not. 

## Written responses

### Written response for Problem 2.A

Put your answer here.

### Written response for Problem 2.B

Put your answer here.

### Written response for Problem 2.C

Put your answer here.